In [1]:
# preprocessing imports

from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import class_weight

# ARCHITECTURE imports

import tensorflow as tf
import numpy as np
from tensorflow.keras import models, Sequential, layers, regularizers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# COMPILATION imports

from tensorflow.keras import optimizers, callbacks

# FIT imports

from tensorflow.keras import callbacks

In [2]:
# preprocessing stuff

# load the numpy arrays
y  = np.load('../malaria/y_values.npy')
X  = np.load('../malaria/X_values.npy')

# one hot encoding y
ohe = OneHotEncoder(sparse_output=False)
ohe.fit(y.reshape(-1,1))
encoded_y = ohe.transform(y.reshape(-1,1))
print(encoded_y)
print(np.unique(y))
# reshaping X
X = np.expand_dims(X,axis=3)
image_shape = (X.shape[1], X.shape[2], X.shape[3])
print(image_shape)

# normalising X
norm_X = X/255 
# print(norm_X)

# class weighting
class_weight_values = class_weight.compute_class_weight('balanced',
                            classes = np.unique(y).tolist(),
                           y=y.reshape(-1))
print(class_weight_values)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
['difficult' 'gametocyte' 'leukocyte' 'red blood cell' 'ring' 'schizont'
 'trophozoite']
(108, 108, 1)
[28.57142857 47.61904762 71.42857143  0.14773231 28.57142857 71.42857143
  8.92857143]


In [3]:
print(type(class_weight_values))
print(type(encoded_y))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [4]:
class_weight_dict = dict(zip(np.unique(y),list(range(len(np.unique(y))))))
class_weight_dict

{'difficult': 0,
 'gametocyte': 1,
 'leukocyte': 2,
 'red blood cell': 3,
 'ring': 4,
 'schizont': 5,
 'trophozoite': 6}

In [5]:
class_weight_values_dict = dict(zip(list(class_weight_dict.values()),class_weight_values))
class_weight_values_dict

{0: 28.571428571428573,
 1: 47.61904761904762,
 2: 71.42857142857143,
 3: 0.14773230905599055,
 4: 28.571428571428573,
 5: 71.42857142857143,
 6: 8.928571428571429}

In [6]:
# model 1

#### 1. ARCHITECTURE

model = Sequential()

model.add(Conv2D(filters=64, kernel_size=(4,4), input_shape=image_shape, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.L1(0.01)))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(filters=32, kernel_size=(2,2), activation='relu', kernel_regularizer=tf.keras.regularizers.L1(0.01)))
# model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(64, activation='relu'))

# Dropouts reduce overfitting by randomly turning neurons off during training.

model.add(Dropout(0.2))

# categorical = softmax
model.add(Dense(7, activation='softmax'))

In [7]:
#### 2. COMPILATION

adam = optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999)

model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

In [8]:
#### 3. FIT

es = callbacks.EarlyStopping(patience=10, restore_best_weights=True)

model.fit(X, encoded_y,
          batch_size=32, # Batch size -too small--> no generalization
          epochs=100,    #            -too large--> slow computations
          validation_split=0.3,
          callbacks=[es],
          class_weight=class_weight_values_dict,
          verbose=1)

Epoch 1/100


2023-03-08 16:39:21.738102: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


22/22 [==============================] - 2s 74ms/step - loss: 8309.3574 - accuracy: 0.3000 - val_loss: 2.0079 - val_accuracy: 0.0133
Epoch 2/100
22/22 [==============================] - 1s 67ms/step - loss: 22.2373 - accuracy: 0.0171 - val_loss: 2.0227 - val_accuracy: 0.0133
Epoch 3/100
22/22 [==============================] - 2s 68ms/step - loss: 101.7409 - accuracy: 0.0029 - val_loss: 2.0119 - val_accuracy: 0.0000e+00
Epoch 4/100
22/22 [==============================] - 2s 70ms/step - loss: 1.9913 - accuracy: 0.0071 - val_loss: 1.9997 - val_accuracy: 0.0133
Epoch 5/100
22/22 [==============================] - 1s 67ms/step - loss: 1.9875 - accuracy: 0.0057 - val_loss: 1.9978 - val_accuracy: 0.0000e+00
Epoch 6/100
22/22 [==============================] - 1s 66ms/step - loss: 1.9853 - accuracy: 0.0029 - val_loss: 2.0012 - val_accuracy: 0.0000e+00
Epoch 7/100
22/22 [==============================] - 1s 66ms/step - loss: 1.9855 - accuracy: 0.0029 - val_loss: 1.9964 - val_accuracy: 0.0000e